##### 101 Formulaic Alphas
- Based on [101 Formulaic Alphas](https://arxiv.org/pdf/1601.00991.pdf), Zura Kakushadze, arxiv, 2015

##### Imports & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.feature_selection \
        import mutual_info_regression
        
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [3]:
idx= pd.IndexSlice
sns.set_style('whitegrid')

##### Loading Data

In [4]:
DATA_STORE = Path('/home/sayem/Desktop/Project/data/assets.h5')

In [5]:
ohlcv = ['open', 'high', 'low', 'close', 'volume']

with pd.HDFStore(DATA_STORE) as store:
    data = (store['data/top500_dataset']
            .loc[:, ohlcv + ['ret_01d', 'sector', 'ret_fwd_01d']]
            .rename(columns={'ret_01d': 'returns'})
            .sort_index())

In [6]:
adv20 = data.groupby('ticker').rolling(20).volume.mean().reset_index(0, drop=True)
data = data.assign(adv20=adv20)
data = data.join(data.groupby('date')[ohlcv].rank(axis=1, pct=True), rsuffix='_rank')
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1355775 entries, ('AA', Timestamp('2013-01-03 00:00:00')) to ('ZTS', Timestamp('2023-08-10 00:00:00'))
Data columns (total 14 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   open         1355775 non-null  float64
 1   high         1355775 non-null  float64
 2   low          1355775 non-null  float64
 3   close        1355775 non-null  float64
 4   volume       1355775 non-null  float64
 5   returns      1355273 non-null  float64
 6   sector       1355775 non-null  float64
 7   ret_fwd_01d  1355775 non-null  float64
 8   adv20        1346239 non-null  float64
 9   open_rank    1355775 non-null  float64
 10  high_rank    1355775 non-null  float64
 11  low_rank     1355775 non-null  float64
 12  close_rank   1355775 non-null  float64
 13  volume_rank  1355775 non-null  float64
dtypes: float64(14)
memory usage: 150.1+ MB


In [7]:
data = data[~data.index.duplicated(keep='first')]
o = data.open.unstack('ticker')
h = data.high.unstack('ticker')
l = data.low.unstack('ticker')
c = data.close.unstack('ticker')
v = data.volume.unstack('ticker')
vwap = o.add(h).add(l).add(c).div(4)
adv20 = v.rolling(20).mean()
r = data.returns.unstack('ticker')

In [8]:
from formulaic_alphas import alpha001

In [9]:
# Applying the alpha001 function
alpha_values = alpha001(c, r)

alpha_values

ticker,AA,AAL,AAP,AAPL,ABBV,ABC,ABEV,ABT,ACN,ADBE,...,WU,WY,WYNN,X,XEL,XOM,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-07,-0.1920,-0.3830,-0.1920,-0.3830,-0.3830,-0.1245,-0.2465,-0.383,-0.0465,-0.0465,...,-0.2465,-0.3830,-0.0465,-0.0465,-0.1245,-0.2465,-0.1920,-0.2465,-0.3830,-0.3830
2023-08-08,-0.0995,-0.4435,-0.0995,-0.4435,-0.2800,-0.0305,-0.1560,-0.280,-0.2800,-0.2800,...,-0.1560,-0.4435,-0.0995,-0.2800,-0.0305,-0.4435,-0.0995,-0.1560,-0.4435,-0.4435
2023-08-09,-0.0390,-0.3610,-0.0390,-0.3610,-0.4475,-0.2190,-0.0940,-0.219,-0.2190,-0.2190,...,-0.0940,-0.4475,-0.0390,-0.2190,-0.4475,-0.4475,-0.0390,-0.0940,-0.3610,-0.4475


In [11]:
alpha = 1

alphas[f'alpha_{alpha:03}'] = alpha001(c, r)

NameError: name 'alphas' is not defined